In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

import gridstatus

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [4]:
ISOS = ["PJM"]

In [5]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

16it [00:02,  7.37it/s]


In [6]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

PJM


In [7]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [8]:
for iso in ISOS:
    clean_final(iso, "final")

                           dasr_procurement_price  dasr_procurement_quantity  \
time_tomerge                                                                   
2014-01-01 05:00:00+00:00                     0.0                     5840.1   
2014-01-01 06:00:00+00:00                     0.0                     5846.7   
2014-01-01 07:00:00+00:00                     0.0                     5477.1   
2014-01-01 08:00:00+00:00                     0.0                     6297.9   
2014-01-01 09:00:00+00:00                     0.0                     5476.9   
...                                           ...                        ...   
2019-01-01 00:00:00+00:00                     0.0                     5223.8   
2019-01-01 01:00:00+00:00                     0.0                     5253.8   
2019-01-01 02:00:00+00:00                     0.0                     5223.8   
2019-01-01 03:00:00+00:00                     0.0                     5223.8   
2019-01-01 04:00:00+00:00               

In [26]:
#FOR COMBING DATA
ISO_done = ["PJM"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/PJM/new"
for file in (listdir_nohidden(FILE_PATH)):
    if file in ["generation"]:
        dfs = []
        for f in tqdm(listdir_nohidden(f"{FILE_PATH}/{file}")):
            df = pd.read_csv(f"{FILE_PATH}/{file}/{f}")
            #df['Date'] = [f[14:24].replace("_", "-") for _ in range(len(df))]
            dfs.append(df)

        temp = pd.concat(dfs)
        os.makedirs(f"{FILE_PATH}/combined", exist_ok=True)
        temp.to_csv(f"{FILE_PATH}/combined/{file}.csv")

4it [00:03,  1.07it/s]


In [9]:
ISO_final[ISO].columns


Index(['dasr_procurement_price', 'dasr_procurement_quantity', 'dasr_cost',
       'rto_activation_cost_nonsynchr', 'mad_activation_cost_nonsynchr',
       'mad_pr_price', 'mad_sr_price', 'rto_mileage_ratio', 'rto_pr_price',
       'rto_reg_capability_price', 'rto_reg_performance_price', 'rto_sr_price',
       'mad_pr_reg', 'mad_sr_req', 'rto_pr_req', 'rto_reg_req', 'rto_sr_req',
       'mad_pr_total', 'mad_sr_total', 'rto_pr_total', 'rto_reg_total',
       'rto_sr_total', 'rto_reg_activation_quantity', 'energy_da', 'lmp_da',
       'congestion_da', 'losses_da', 'energy_rt', 'lmp_rt', 'congestion_rt',
       'losses_rt', 'exchange_rt', 'unnamed: 0', 'gas_price', 'coal', 'gas',
       'hydro', 'multiple fuels', 'nuclear', 'oil', 'other',
       'other renewables', 'solar', 'wind', 'load', 'gen_wind'],
      dtype='object')

In [27]:
ISO_done = ["PJM"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"
new_data = {}
for ISO in ISO_done:
    sub_folder = f"{FILE_PATH}/{ISO}/new/combined"
    files = [f for f in listdir(sub_folder) if isfile(join(sub_folder, f)) and f[0]!="."]
    new_data[ISO] = {}
    print(ISO)
    for file in tqdm(files):
        new_data[ISO][file.replace(".csv", "")] = pd.read_csv(f"{sub_folder}/{file}")

PJM


100%|██████████| 11/11 [00:03<00:00,  2.77it/s]


In [28]:
print([key for key in new_data["PJM"]])

['solar', 'ancillary_dasr_procurement', 'energy_da', 'ancillary_procurement_price', 'energy_rt', 'wind', 'generation', 'ancillary_nonsynchr_cost', 'load', 'exchange', 'ancillary_procurement_quantity']


In [46]:
energy_rt = new_data["PJM"]["energy_rt"]
energy_rt = energy_rt[energy_rt["pnode_name"] == "PJM-RTO"]
energy_rt["time_tomerge"] = pd.to_datetime(energy_rt["datetime_beginning_ept"]) + timedelta(hours=1)
energy_rt = energy_rt[["time_tomerge", "system_energy_price_rt", "congestion_price_rt", "marginal_loss_price_rt"]].rename(columns={"system_energy_price_rt": "energy_rt", "congestion_price_rt": "congestion_rt", "marginal_loss_price_rt": "losses_rt"})
print(energy_rt)

              time_tomerge  energy_rt  congestion_rt  losses_rt
0      2022-01-01 01:00:00      18.91       0.000251   0.006118
23     2022-01-01 02:00:00      18.16       0.000000   0.005346
46     2022-01-01 03:00:00      18.50       0.000162   0.005814
69     2022-01-01 04:00:00      18.38       0.000711   0.009022
92     2022-01-01 05:00:00      18.11      -0.000033   0.006453
...                    ...        ...            ...        ...
811992 2019-12-30 21:00:00      18.69       0.010342   0.008448
812015 2019-12-30 22:00:00      18.50       0.007895   0.008187
812038 2019-12-30 23:00:00      17.62       0.006954   0.011930
812061 2019-12-31 00:00:00      17.47       0.009210   0.010926
812084 2019-12-31 01:00:00      17.41       0.052813   0.009547

[35309 rows x 4 columns]


/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_62008/2457076408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_rt["time_tomerge"] = pd.to_datetime(energy_rt["datetime_beginning_ept"]) + timedelta(hours=1)


In [48]:
energy_da = new_data["PJM"]["energy_da"]
energy_da = energy_da[energy_da["pnode_name"] == "PJM-RTO"]
energy_da["time_tomerge"] = pd.to_datetime(energy_da["datetime_beginning_ept"]) + timedelta(hours=1)
energy_da = energy_da[["time_tomerge", "system_energy_price_da", "congestion_price_da", "marginal_loss_price_da"]].rename(columns={"system_energy_price_da": "energy_da", "congestion_price_da": "congestion_da", "marginal_loss_price_da": "losses_da"})
print(energy_da)

              time_tomerge  energy_da  congestion_da  losses_da
0      2019-01-01 01:00:00      18.96      -0.017594   0.048129
23     2019-01-01 02:00:00      18.80      -0.025540  -0.008323
46     2019-01-01 03:00:00      18.57      -0.014702  -0.027308
69     2019-01-01 04:00:00      18.40      -0.016699  -0.018047
92     2019-01-01 05:00:00      18.57      -0.010931  -0.023695
...                    ...        ...            ...        ...
804264 2020-12-30 21:00:00      22.03       0.125627   0.266690
804287 2020-12-30 22:00:00      20.53       0.058729   0.228399
804310 2020-12-30 23:00:00      18.78       0.007678   0.117070
804333 2020-12-31 00:00:00      18.10       0.070194   0.130718
804356 2020-12-31 01:00:00      17.25       0.329560   0.024265

[34973 rows x 4 columns]


/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_62008/811313519.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_da["time_tomerge"] = pd.to_datetime(energy_da["datetime_beginning_ept"]) + timedelta(hours=1)


In [52]:
load = new_data["PJM"]["load"]
load["time_tomerge"] = pd.to_datetime(load["datetime_beginning_ept"]) + timedelta(hours=1)
load = load[["time_tomerge", "mw"]].rename(columns={"mw":"load"})
print(load)

             time_tomerge       load
0     2020-01-01 01:00:00  79182.471
1     2020-01-01 02:00:00  77573.714
2     2020-01-01 03:00:00  76202.136
3     2020-01-01 04:00:00  75674.034
4     2020-01-01 05:00:00  76238.041
...                   ...        ...
35059 2019-12-31 20:00:00  92784.789
35060 2019-12-31 21:00:00  90016.822
35061 2019-12-31 22:00:00  87332.351
35062 2019-12-31 23:00:00  84430.223
35063 2020-01-01 00:00:00  81522.416

[35064 rows x 2 columns]


In [78]:
exchange = new_data["PJM"]["exchange"]
exchange_average = pd.DataFrame()
exchange_average["time_tomerge"] = np.unique(pd.to_datetime(exchange["datetime_ending_ept"]))
exchange_average = exchange_average.merge(exchange[exchange["hrly_net_tie_sched"]>0].groupby(["time_tomerge"])["hrly_net_tie_sched"].sum(), on = "time_tomerge").rename(columns = {"hrly_net_tie_sched": "exchange_positive"})
exchange_average = exchange_average.merge(exchange[exchange["hrly_net_tie_sched"]<0].groupby(["time_tomerge"])["hrly_net_tie_sched"].sum(), on = "time_tomerge").rename(columns = {"hrly_net_tie_sched": "exchange_negative"})
exchange_average = exchange_average.merge(exchange.groupby(["time_tomerge"])["hrly_net_tie_sched"].sum(), on = "time_tomerge").rename(columns = {"hrly_net_tie_sched": "exchange"})
exchange_average

,time_tomerge,exchange_positive,exchange_negative,exchange
0,2019-01-01 01:00:00,1273.75,-4706.00,-3432.25
1,2019-01-01 02:00:00,1582.00,-5814.50,-4232.50
2,2019-01-01 03:00:00,1656.75,-5607.25,-3950.50
3,2019-01-01 04:00:00,1668.25,-5574.00,-3905.75
4,2019-01-01 05:00:00,1455.00,-5908.00,-4453.00
...,...,...,...,...
32856,2022-12-30 21:00:00,982.00,-3632.25,-2650.25
32857,2022-12-30 22:00:00,771.00,-3568.75,-2797.75
32858,2022-12-30 23:00:00,1152.50,-3669.00,-2516.50
32859,2022-12-31 00:00:00,1587.50,-3638.75,-2051.25


In [81]:
generation = new_data["PJM"]["generation"]
generation["time_tomerge"] = pd.to_datetime(generation["datetime_beginning_ept"]) + timedelta(hours=1)


In [88]:
generation = generation.reset_index()

In [ ]:
pd.pivot(generation, index="index", columns=["fuel_type", "time_tomerge"], values="mw")

In [89]:
print(generation)

         index  Unnamed: 0 datetime_beginning_utc  datetime_beginning_ept  \
0            0           0  12/31/2022 5:00:00 AM  12/31/2022 12:00:00 AM   
1            1           1  12/31/2022 5:00:00 AM  12/31/2022 12:00:00 AM   
2            2           2  12/31/2022 5:00:00 AM  12/31/2022 12:00:00 AM   
3            3           3  12/31/2022 5:00:00 AM  12/31/2022 12:00:00 AM   
4            4           4  12/31/2022 5:00:00 AM  12/31/2022 12:00:00 AM   
...        ...         ...                    ...                     ...   
384607  384607       96091    1/1/2019 5:00:00 AM    1/1/2019 12:00:00 AM   
384608  384608       96092    1/1/2019 5:00:00 AM    1/1/2019 12:00:00 AM   
384609  384609       96093    1/1/2019 5:00:00 AM    1/1/2019 12:00:00 AM   
384610  384610       96094    1/1/2019 5:00:00 AM    1/1/2019 12:00:00 AM   
384611  384611       96095    1/1/2019 5:00:00 AM    1/1/2019 12:00:00 AM   

               fuel_type       mw  fuel_percentage_of_total  is_renewable  